# **OCR-Based-Document-Data-Extraction-System**

1. Mount Google Drive

In [1]:
from google.colab import drive

# Mount the Google Drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


2. Install the necessary libraries

In [3]:
!pip install opencv-python-headless pandas openpyxl

In [6]:
!apt-get install -y tesseract-ocr
!apt-get install -y libtesseract-dev

Reading package lists... Done
Building dependency tree... Done
Reading state information... Done
tesseract-ocr is already the newest version (4.1.1-2.1build1).
0 upgraded, 0 newly installed, 0 to remove and 49 not upgraded.
Reading package lists... Done
Building dependency tree... Done
Reading state information... Done
libtesseract-dev is already the newest version (4.1.1-2.1build1).
0 upgraded, 0 newly installed, 0 to remove and 49 not upgraded.


In [5]:
import os
import cv2
import pandas as pd
import json

3. Define the necessary paths

In [6]:
images_folder = '/content/drive/MyDrive/Images'  # Path to images folder
json_file_path = '/content/drive/MyDrive/filtered-data.json'  # Path to JSON file
output_csv = '/content/Output.csv'  # Output CSV file path

4. Load JSON data and define Template texts

In [25]:
# Load JSON data
with open(json_file_path, 'r') as json_file:
    data = json.load(json_file)

# Template text (can be expanded)(Hardcoded)
template_texts = ['Republic','of', 'Estonia', 'Identity','Card', 'EESTI', 'VABARIK', 'ISIKUTUNNISTUS']

# Initialize lists to store data
rows = []

5. Iterate through the JSON data and the images provided

In [41]:
# Iterate through the JSON data
for idx, doc in data.items():
    image_filename = doc.get("path", "")
    doc_type = doc.get("type", "")
    size = doc.get("size", [0, 0])
    ocr_data = doc.get("ocr", {})

    # Construct full image path
    image_path = os.path.join(images_folder, image_filename + '.jpeg')
    if not os.path.exists(image_path):
        print(f"Image not found: {image_path}")
        continue

    # Load image
    image = cv2.imread(image_path)
    if image is None:
        print(f"Failed to load image: {image_path}")
        continue

        # Process OCR data from JSON
for text, coords in ocr_data.items():
    # Determine label
    label = 'unknown' if text in template_texts else 'user_data'

    # Extract box coordinates from JSON and normalize them
    x_coords = [c['x'] for c in coords]
    y_coords = [c['y'] for c in coords]

    # Calculate box dimensions (in original scale)
    x_min, x_max = min(x_coords) * size[0], max(x_coords) * size[0]
    y_min, y_max = min(y_coords) * size[1], max(y_coords) * size[1]
    width = x_max - x_min
    height = y_max - y_min

    # Calculate width and height ratios
    width_ratio = width / size[0] if size[0] else 0
    height_ratio = height / size[1] if size[1] else 0

    # Append the extracted data to the 'rows' list
    rows.append({
        "Document Path": image_path,
        "Label": label,
        "Text": text,
        "X Coordinates": x_coords,
        "Y Coordinates": y_coords,
        "Width": width,
        "Height": height,
        "Width Ratio": width_ratio,
        "Height Ratio": height_ratio,
    })

In [42]:
# Create a DataFrame
df = pd.DataFrame(rows)

# Save to CSV
df.to_csv(output_csv, index=False)

print(f"Preprocessing completed. Data saved to {output_csv}")

Preprocessing completed. Data saved to /content/Output_assessment.csv
